# Build a Causal Language Model Using PyTorch
In notebook07 we learned how to build a causal language model (GPT) from scratch. In this homework, we will build a Causal Language model with PyTorch, which also supports back-propagation compared to the notebook implementation.

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

We are going to train our GPT on this corpus

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-16 12:26:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-04-16 12:26:59 (19.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



For simplicity, character-level tokenization is applied here instead of learning a new BPE tokenizer

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


### TODO1: Single-head Attention Implementation

In [5]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # register_buffer simply adds self.tril as a non-paramater buffer in this nn.Module
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # TODO: fill in the forward pass for single-head attention
        _,t,c = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        # verbose names for my own understanding
        scaled_attention = q @ k.transpose(-2,-1) * c**-0.5
        causal_scaled_attention = scaled_attention.masked_fill(self.tril[:t, :t] == 0, float('-inf'))
        normalised_causal_scaled_attention = F.softmax(causal_scaled_attention, dim=-1)
        dropout_normalised_causal_scaled_attention = self.dropout(normalised_causal_scaled_attention)
        return dropout_normalised_causal_scaled_attention @ v

### Multi-head attention and feed forward layer implementations

In [6]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### TODO2: Transformer Block Implementation

In [7]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # TODO: fill in the forward pass
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        # Hint: don't forget the residual connections
        return x

### Language Model Implementation.

In [8]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [20]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = LanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.3777, val loss 4.3841
step 100: train loss 2.6692, val loss 2.6921
step 200: train loss 2.5086, val loss 2.5197
step 300: train loss 2.4133, val loss 2.4310
step 400: train loss 2.3655, val loss 2.3799
step 500: train loss 2.2960, val loss 2.3238
step 600: train loss 2.2490, val loss 2.2725
step 700: train loss 2.1979, val loss 2.2205
step 800: train loss 2.1682, val loss 2.2012
step 900: train loss 2.1183, val loss 2.1532
step 1000: train loss 2.0870, val loss 2.1227
step 1100: train loss 2.0516, val loss 2.0994
step 1200: train loss 2.0114, val loss 2.0872
step 1300: train loss 2.0029, val loss 2.0622
step 1400: train loss 1.9754, val loss 2.0327
step 1500: train loss 1.9545, val loss 2.0235
step 1600: train loss 1.9321, val loss 2.0108
step 1700: train loss 1.9130, val loss 1.9940
step 1800: train loss 1.8858, val loss 1.9841
step 1900: train loss 1.8772, val loss 1.9691
step 2000: train loss 1.8507, val loss 1.9828
step 2100: train loss 1.

### TODO3: BERTopic Practice
Use BERTopic to obtain top-5 topics in AG's news dataset

In [11]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

--2024-04-16 12:45:59--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M   146MB/s    in 0.2s    

2024-04-16 12:45:59 (146 MB/s) - ‘train.csv’ saved [29470338/29470338]



In [12]:
import pandas as pd
df = pd.read_csv("train.csv", header=None)

# Analyze the topic distribution of first 5000 news leading sentences
docs = df.iloc[:, 2].tolist()
docs = docs[:5000]

In [15]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.5 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (73

In [16]:
# TODO: conduct topic modeling following the steps of the last section in https://github.com/elliottash/lm_lss_2024/blob/main/notebooks/08_LLMs.ipynb
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)

2024-04-16 12:54:40,983 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2024-04-16 12:57:29,550 - BERTopic - Embedding - Completed ✓
2024-04-16 12:57:29,552 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-16 12:58:11,189 - BERTopic - Dimensionality - Completed ✓
2024-04-16 12:58:11,195 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-16 12:58:11,541 - BERTopic - Cluster - Completed ✓
2024-04-16 12:58:11,561 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-16 12:58:12,367 - BERTopic - Representation - Completed ✓


In [17]:
model.get_topic(1)

[('google', 0.04650215348748798),
 ('offering', 0.03183425145505234),
 ('initial', 0.031487460845556786),
 ('public', 0.03146234675973264),
 ('search', 0.02611388341016574),
 ('ipo', 0.023716588435029284),
 ('stock', 0.0226177851707973),
 ('inc', 0.019270057164044913),
 ('shares', 0.018298428001723),
 ('price', 0.018191550957310326)]

In [18]:
model.get_topic(0)

[('night', 0.015602287853145663),
 ('season', 0.01372072610978267),
 ('his', 0.013589219578398409),
 ('ap', 0.012807777115133607),
 ('league', 0.012177341385398605),
 ('inning', 0.011997679657857587),
 ('sox', 0.011087085406653532),
 ('the', 0.010214330158058482),
 ('baseball', 0.009775562291574582),
 ('win', 0.009652375596236122)]

In [19]:
model.get_topic(2)

[('najaf', 0.07284794434436874),
 ('cleric', 0.0479069295839937),
 ('shiite', 0.04393130336889755),
 ('iraqi', 0.041159032596604875),
 ('alsadr', 0.040522142641702966),
 ('radical', 0.04037810300681909),
 ('iraq', 0.03547497127242664),
 ('shrine', 0.032482663768332545),
 ('holy', 0.03176800176998323),
 ('baghdad', 0.02645435140681235)]

In [ ]:
!jupyter nbconvert